# Singular Value Decomposition of EW and Spectra

Import the used packages

Make sure that we don't get an error when we call pyraf. It is usually that we need to create a uparm folder and correctly initialize pyraf from the command line. 

In [2]:
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Span, Label, Arrow, NormalHead
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
from bokeh.layouts import  column, row
from bokeh.palettes import Category20_18
import re
import glob, os
from astropy.io import fits
import urllib
#from urllib import urlretrieve
from sklearn.decomposition import TruncatedSVD
#Garbage collector
import gc

from pyraf import iraf
output_notebook()
iraf.noao.onedspec()
iraf.dataio()

Loading BokehJS ...

Defined the classes. Lines and spec. Add a new attribute with the general type of star. 

In [3]:
#Emission Lines
class line(object):
    """Line with the name of the line, line center and the region around it. 
    This region around it will be use to fit it using IRAF function. It has to be a list of numbers"""
    def __init__(self,name,linecenter,regiontofit):
        self.name = name
        self.linecenter = linecenter
        self.regiontofit = regiontofit
        
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx, array[idx]

def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
    
def gaussian(x, mu, sig,core):
    """Gaussian"""
    return core*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

        
class spec2(object):
    """Spectra created from the url. Probably in the future better with the fiber
    , plate and MJD"""
    def __init__(self,url):
        self.url = url
        self.name = url.split('/')[-1]
        self.wave, self.flux, self.model, self.type, self.typegeneral = self.get_type()
       
            
    def get_type(self, verbose=False):
        """Get the type, flux, wavelenght and model from the SDSS TABLE. 
        It downloadst he fit if it doesnt exist. Had to do it like that becase it would waint until the download
        to call the other method"""
        #Download if it doesnt't exists. 
        if not os.path.isfile(self.name):
            file_name = self.name
            u = urllib.urlopen(self.url)
            f = open(file_name, 'wb')
            meta = u.info()
            file_size = int(meta.getheaders("Content-Length")[0])
            #print "Downloading: %s Bytes: %s" % (file_name, file_size)

            file_size_dl = 0
            block_sz = 8192
            while True:
                buffer = u.read(block_sz)
                if not buffer:
                    break

                file_size_dl += len(buffer)
                f.write(buffer)
                status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
                status = status + chr(8)*(len(status)+1)
                if verbose:
                    print status,

            f.close()
            #print('Downloaded '+ self.name)
        
        ob = fits.open(self.name,memmap=False)
        #Get data and save wavelenft in x, and flux on y
        dataob = ob[1].data
        x = 10**(dataob['loglam'])
        y2 = dataob['model']
        y = dataob['flux']
        typee = ob[2].data['SUBCLASS'][0]
        ob.close()
        #Delete
        del ob
        del dataob
        gc.collect()
        
        npattern = re.compile('[O,B,A,F,G,K,WD,M,CV]{1,2}')
        typegeneral = npattern.findall(typee)
        if 'WD' in typegeneral:
            typegeneral = 'WD'
        elif len(typegeneral) == 0:
            typegeneral = 'Other'
        else:
            typegeneral = typegeneral[0]
        #delete to aboid too many oepenf files see
        #http://docs.astropy.org/en/stable/io/fits/appendix/faq.html#i-m-opening-many-fits-files-in-a-loop-and-getting-oserror-too-many-open-files
        return x,y,y2, typee, typegeneral
    
    def get_iraffits(self, modelfit = True):
        """From the SDSS table creates a fits file to work with iraf"""
        #Which one to convert to iraf fts
        if modelfit:
            fluxormodel = self.model
        else:
            fluxormodel = self.flux
        #Name of iraf .txt file and create text file
        
        namenewfits = '{name}.txt'.format(name=self.name)
        with open(namenewfits,'w') as file:
            for x,y in zip(self.wave,fluxormodel):
                file.write('{}\t{}\n'.format(x,y))
                
        #Create fits file from text file interpolation to work wtih IRAF functions
        iraffitsname = 'iraf'+self.name
        if os.path.exists(iraffitsname+'.fits'):
            os.remove(iraffitsname+'.fits')
        iraf.rspectext(namenewfits,iraffitsname,dtype='interp')
        #Remove the text file
        os.remove(namenewfits)
        
    
    def fit_lines(self, dicoflines, errorestimate = True, verbose='No'):
        """Fit lines using gaussian fitport. It populates a diccionary with the lines
         and the fit of the lines. The parameter if a diccionary of lines objects. 
         The class is define before. Verbose can be yes or NO"""
        self.linesdicall = []
        errorparam = []
        if not os.path.exists('iraf'+self.name):
            self.get_iraffits()
            
        #Initialize files log and lines
        ! echo '' > fited.log

        for indexline,linesfit in enumerate(dicoflines):
            regionf = "{} {}".format(linesfit.regiontofit[0],linesfit.regiontofit[1])
            #wavelenght
            xlimns = [find_nearest(self.wave,i)[0] for i in linesfit.regiontofit   ]
            wavex = self.wave[xlimns[0]:xlimns[1]]
            lineszero = linesfit.linecenter
            ! echo '$lineszero' > lines.lines

            filename = 'iraf'+self.name
            #Error estimation iraf: http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?fitprofs
            if errorestimate:        
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log'
                              ,nerrsample='100',sigma0='4',invgain='4',verbose=verbose)
            else:
                iraf.fitprofs(filename,pos='lines.lines', reg=regionf ,
                              fitbackground= 'yes', 
                              logfile='fited.log', verbose=verbose)

            #Plotting the gaussian
            #Find in log file of fitprofs.  
            npattern = re.compile('[-\d.]+')
            npattern2 = re.compile('[-\d.E?]+') #Gets the exponentioals

            gparameters=[]
            with open('fited.log','r') as file:
                for lines in file:

                    if '(' not in lines:
                        temp = npattern2.findall(lines)
                        if 'INDEF' in lines:
                            gparameters.append(7*[0])
                    else:
                        errorparam.append(npattern2.findall(lines))

                    if len(temp) == 7 and all(isDigit(i) for i in temp) and 'INDEF' not in lines:
                        gparameters.append(temp)

            #gaussian
            if len(gparameters) > 0:
                gparamfinal = [ float(i) for i in gparameters[-1] ]
                centerg, contg, fluxg, eqwg, coreg, fwhmg, fwhml = gparamfinal
                yg = gaussian(wavex,centerg,fwhmg/2.3538,coreg) + contg


            if errorestimate == True:
                errorparamfinal = [ float(i) for i in errorparam[-1] ]
                #print(gparamfinal)
                #print(errorparamfinal)


                linesdic = {'linename':linesfit.name,
                             'center':centerg,
                              'EW': eqwg,
                              'EWerror':errorparamfinal[3],
                            'fluxg':fluxg,
                            'coregaus':coreg,
                            'fwgmgaus':fwhmg,
                            'contgaus':contg,
                            'gaussian':{'x':wavex,'y':yg}
                            }
            else:
                 linesdic = {'linename':linesfit.name,
                             'center':centerg,
                              'EW': eqwg,
                              'EWerror':0,
                            'fluxg':fluxg,
                            'coregaus':coreg,
                            'fwgmgaus':fwhmg,
                            'contgaus':contg,
                            'gaussian':{'x':wavex,'y':yg}
                            }
                
                
                
            self.linesdicall.append(linesdic)

        
        


In [4]:
gc.collect()
#Define the new object
tryspec = spec2('https://dr14.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0266/spec-0266-51630-0015.fits')
tryspec.typegeneral

'A'

In [5]:
%%time
diclines = {line('P15',8547,[8520,8560]),line('P14',8600,[8600-20,8600+20]),
           line('Halpha',6562,[6550,6575]),line('P16',8504,[8504-20,8504+20]),
            line('P13',8667,[8667-20,8667+20]),line('P12',8752,[8752-20,8752+20]),
           line('P11',8865,[8865-20,8865+20]),line('P10',9017,[9017-20,9017+20])}




dicofspectra = [];
filestoread = ['../listA1V.txt','../listB1Ve.txt',
              '../listF2V.txt','../listO.txt','../listG1V.txt',
              '../listK0V.txt','../listM0V.txt','../listM2V.txt']

filestoread = glob.glob('../list*.txt')

limitonlines = 10
print('Reading {} files and {} lines per file'.format(len(filestoread),limitonlines))

for index,files in enumerate(filestoread):
    with open(files,'r') as f:
        linereads = f.readlines()
        for i in range(min(limitonlines, len(linereads))):
            lineread = linereads[i]
            #print(lineread)
            url = lineread.split(',')[1]
            #Strip Return a copy of the string with leading and trailing characters removed. 
            url = url.strip()
            url = url.replace('segue1','sdss')
            #print(url)
            tempspec = spec2(url)
            dicofspectra.append(tempspec)
            gc.collect()
            #print(index)
            
            
#Plot parameters
colors = 100*Category20_18 #+ Viridis8 + Dark2_8 + Paired8 + Set1_4
#Plot range
xr = (5000,8900)
yr = (0,40)


#Tool to get wavelength
hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "($x{1}, $y)"),
        ]
    )
#Add the tool
#Start index of color
index = 0
fit = True
plot = False

if plot:
        #Create the Bokeh Figure
    pl =  figure(x_axis_label='Angstrom', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
                  ,active_drag='pan', active_scroll='wheel_zoom',
                  plot_width=900, plot_height=1000
                 )

    pl.add_tools(hover2)





# Loop over spec object list:

for spectra in dicofspectra:

    # Plot the 
    if plot:
        t = pl.line(x=spectra.wave,y=spectra.model,color=colors[index], line_alpha=1.0, 
                line_width=4,legend=spectra.type+str(index),muted_alpha=0.,muted_color=colors[index])

    if fit:
        try:
            spectra.fit_lines(diclines, verbose='No', errorestimate=False)
        except:
            pass
        for i in spectra.linesdicall:
            if plot:
                pl.line(i['gaussian']['x'],i['gaussian']['y'],color='red', line_alpha=1.,
                    line_width=5,legend=tryspec.name,muted_alpha=0.,muted_color=colors[index])

    index = index +1

if plot:    
    pl.legend.location = "top_left"
    pl.legend.click_policy="mute"

    show(pl)  

            

Reading 154 files and 10 lines per file


/home/mmarcano/anaconda3/envs/iraf27/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in divide


In [7]:
a=np.unique([ i.typegeneral for i in dicofspectra], return_counts=True)
a

(array(['A', 'B', 'C', 'CV', 'F', 'G', 'K', 'M', 'O', 'OB', 'Other', 'WD'],
       dtype='|S5'),
 array([188, 262,  20,  10, 210, 141, 200, 220,  35,  10, 100,  60]))

%%time
#Order type of stars and the index that will serve as type to scatter plto
typesAstar, indexAstars = np.unique([i.typegeneral for i in dicofspectra],return_inverse=True)
#Plot range
xr = (0,len(typesAstar))
yr = (0,10)




#Plot
p =  figure(x_axis_label='Type of Star', y_axis_label='Y',title="Click on the desired stellar template to overplot", x_range=xr, y_range=yr
              ,active_drag='pan', active_scroll='wheel_zoom',
              plot_width=900, plot_height=1000
             )


halphalines = []
for number, spectra in enumerate(dicofspectra):
    #IF not lines fitted then try to do it
    if not hasattr(spectra,'linesdicall'):
        try:
            spectra.fit_lines(diclines, verbose='No')
        except:
            pass
    #If it already had the lines them get the information and plot
    if hasattr(spectra,'linesdicall'):
        lines = spectra.linesdicall
        for line in lines:
            if line['linename'] == 'P14':
                stellartype = indexAstars[number]
                #print(spectra.name,line['EW'])
                p.circle(stellartype, line['EW'], 
                         legend = typesAstar[stellartype], color = colors[stellartype], size=10)
                    


show(p)

# Singular Value Decomposition

We build a matrix with all the lines and stars. The rows represent the Equivalent width of an specific line. Rows are equivalent width of lines, and columns each stars. 

In [12]:
for index,i in enumerate(diclines):
    print(index,i.name)

(0, 'P12')
(1, 'P11')
(2, 'P10')
(3, 'P15')
(4, 'P14')
(5, 'Halpha')
(6, 'P16')
(7, 'P13')


We build the matrix. 

In [13]:
matrix = []
#Loop over each line in the diccionart of lines we created. 
for linesfit in diclines:
    listofeq = []
    starclass = []
    #Loop over all the stars in the diccionary of spectra.
    #For each star if we were able to fit a line get the EW
    #Of the line
    for indexs,stars in enumerate(dicofspectra):
        if hasattr(stars,'linesdicall'):
            lines = stars.linesdicall
            #We find the line and ew of the lne and append
            for line in lines:
                if line['linename'] == linesfit.name:
                    eq = line['EW']
                    listofeq.append(eq)
                    starclass.append(stars.typegeneral)
                    
    matrix.append(listofeq)
        #    print(stars.typegeneral)

In [14]:
mat = np.array(matrix)
mat.shape

(8, 1452)

Lets print the first five columns:

In [17]:
for row in mat:
    print(' '.join([str(elem) for elem in row[0:5]]))

17.03 0.331 9754.0 7224.0 3.479
0.936 0.5198 1.045 1.024 1.15
1.826 1.131 1.888 1.825 1.89
0.867 0.7209 0.9137 0.9179 0.7509
0.1015 10420.0 0.1073 0.3124 2430.0
-2.105 -1.413 -2.221 -2.418 -1.137
-0.4023 -0.2569 0.2294 0.2879 2.525
0.692 0.5782 0.7486 0.748 0.6703


We recognize that the fifth columns is the Halpha. We check this by printing the Halpha EW for the first five stars in the diccionary of spectra.

In [18]:
for stars in dicofspectra[0:5]:
    if hasattr(stars,'linesdicall'):
        lines = stars.linesdicall
        #We find the line and ew of the lne and append
        for line in lines:
            if line['linename'] == 'Halpha':
                  print(line['EW'])

-2.105
-1.413
-2.221
-2.418
-1.137


Since we get the expected values we can now we can proceed to do the Singular Value Decomposition of the matrix we just constructed. 

In [19]:
U, D, V = np.linalg.svd(mat, full_matrices=False)
D

array([ 887110.24336337,  246307.53069961,  199295.00741022,
        131160.74439699,  122865.40933372,  112540.64757108,
         97267.46532741,   64259.10148977])

In [20]:
V

array([[ -1.91981424e-05,  -1.54256135e-05,  -1.09952282e-02, ...,
         -8.75442091e-08,  -4.65066621e-05,  -3.86848036e-07],
       [  8.33938997e-06,  -3.20967445e-03,   4.84933984e-05, ...,
         -2.94369314e-06,  -9.85483270e-03,  -1.00690067e-05],
       [ -2.26713240e-06,  -5.14245448e-02,   6.12952035e-05, ...,
         -3.09523753e-07,  -1.57328761e-01,  -1.71054902e-06],
       ..., 
       [ -8.34087924e-06,   2.98427428e-03,  -5.14905613e-06, ...,
         -1.50451041e-06,   8.38982253e-03,  -6.51125765e-06],
       [  5.92056734e-06,  -1.18160435e-03,   5.68446269e-06, ...,
         -5.84515804e-07,  -2.53625255e-02,  -2.54484597e-06],
       [ -2.78738591e-05,   1.25264388e-04,   1.32394051e-04, ...,
         -6.13664970e-06,   2.28864334e-04,  -2.79003293e-05]])

Let's check we can reconstructed the original matrix m. We can use the function allclose that:
"Returns True if two arrays are element-wise equal within a tolerance."

In [21]:
np.allclose(mat, np.dot(U * D, V))

True

In [22]:
coldic ={}
for index,typestar in enumerate(np.unique([i.typegeneral for i in dicofspectra])):
    coldic[str(typestar)] = colors[index]
coldic
label = starclass
colorsvd = [ coldic[i] for i in starclass ]

In [23]:
dd = {}
dd['x'] = V[0]
dd['y'] = V[1]
dd['labels'] = label
dd['colors'] = colorsvd
source = ColumnDataSource(dd)

In [24]:
p = figure()


p.circle(x='x', y='y', color='colors', legend = 'labels',source=source , size =5)
show(p)
#colors
#firstandsecondv = [ xi,j) for i,j in zip(V[0],V[1])  ]
#ColumnDataSource = firstandsecondv

# SVD and PCA of Spectra